# May have to comment out lines 20 and 21. 

### Sympy did not want to be found on my computer so I had to manually add its path.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
# from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.backends.backend_tkagg import ( FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.figure import Figure

from tkinter import *
import tkinter as tk
from tkinter import ttk


import numpy as np
import matplotlib.animation as animation
from scipy.sparse import diags
import time

import sys
sys.path.append('/Applications/anaconda3/pkgs/sympy-1.9-py38h50d1736_0/lib/python3.8/site-packages')
sys.path.append('/Applications/anaconda3/pkgs/mpmath-1.2.1-pyhd8ed1ab_0/site-packages')
import sympy
from sympy.utilities.lambdify import lambdify

# Functions for understanding user defined potential

In [2]:
def is_float(value):
  try:
    float(value)
    return True
  except:
    return False

def Potential_Click2(canvas, ax, *args, **kwargs):
#     x, y, z = sympy.symbols('x y z')
    cmd = kwargs.get('cmd', None)
    min_x = kwargs.get('min_x', None)
    max_x = kwargs.get('max_x', None)
    ax.clear()


    if is_float(min_x):
        min_x = float(min_x)
    else:
        min_x = -10
        
    if is_float(max_x):
        max_x = float(max_x)
    else:
        max_x = 10
        

    
    n = np.linspace(min_x, max_x, 1000)
    
    per_up = 1.05
    per_dn = 0.95
    if max_x > 0:
        fig_max = [max_x, max_x*per_up]
    else:
        fig_max = [max_x, max_x*per_dn]
        
    if min_x > 0:
        fig_min = [min_x*per_dn, min_x]
    else:
        fig_min = [min_x*per_up, min_x]
    
    x = sympy.symbols('x')

    func = lambdify(x, cmd,'numpy')
    numpy_array_of_results = func(n)
    ax.plot(n, numpy_array_of_results)
    ax.set_xlim(fig_min[0], fig_max[1])
    ax.axvspan(fig_min[0], fig_min[1], alpha=0.1, color='green')
    ax.axvspan(fig_max[0], fig_max[1], alpha=0.1, color='green')
    canvas.draw()
    

# Function for numerically solving the system

### Change the `def initial_state` (initial state  of particle)
### Change mass, $\Delta t$ and $\Delta x$ in `def animation_Click` (GUI functionality not set up yet)


# WARNING: Animation saves itself to your system as 'animation.mp4' line 140, and then opens the file to show the user. Poor implementation... but it works

In [3]:
def initial_state(x, x0, p0, sigma, h):
    return 1/(np.pi**0.25*sigma**0.5)*np.exp(-((x-x0)**2/(2*sigma**2)))*np.exp(1j*p0*x/h)


def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def fun_A(x, time, dx, dt, m, h, V):
    return  - 2 + (4j*m*dx**2)/(h*dt) - ((2*m*dx**2)/(h**2))*V

def fun_B(x, time, psi, dx, dt, m, h, V):
    return -psi[0:len(x)-2] - psi[2:len(x)] + psi[1:len(x)-1]*(2 + (4j*m*dx**2)/(h*dt) + ((2*m*dx**2)/(h**2))*V)



# def V(x, t):
#     '''
#     Potential
#     '''
#     return 0 + 0j


def Animation_Click(canvas, ax, *args, **kwargs):

    cmd = kwargs.get('cmd', None)
    min_x = kwargs.get('min_x', None)
    max_x = kwargs.get('max_x', None)
    ax.clear()
        
    data_points = 1000
    nx = data_points + 1

    dt = 0.1
    time_points = 100
    
    t = np.arange(0, time_points*dt, dt)

    dx = 0.01
    x = np.arange(0, nx*dx, dx)
    
    m = 5.68
    h = 0.6582
    x0 = nx*dx/2
    p0 = nx*dx/10
    sigma = nx*dx/20
    
    if is_float(min_x):
        min_x = float(min_x)
    else:
        min_x = 0
        
    if is_float(max_x):
        max_x = float(max_x)
    else:
        max_x = nx*dx
    
#     n = np.linspace(min_x, max_x, 10)
    
    per_up = 1.05
    per_dn = 0.95
    if max_x > 0:
        fig_max = [max_x, max_x*per_up]
    else:
        fig_max = [max_x, max_x*per_dn]
        
    if min_x > 0:
        fig_min = [min_x*per_dn, min_x]
    else:
        fig_min = [min_x*per_up, min_x]
#     
    x_sym = sympy.symbols('x')

    V = lambdify(x_sym, cmd,'numpy')
    numpy_array_of_results = V(x)
    ax.plot(x, numpy_array_of_results)
    ax.set_xlim(fig_min[0], fig_max[1])
    ax.axvspan(fig_min[0], fig_min[1], alpha=0.1, color='green')
    ax.axvspan(fig_max[0], fig_max[1], alpha=0.1, color='green')
    canvas.draw()



    psi = np.zeros((len(t), len(x))) + 0j
    # print("PSI shape = " + str(np.shape(psi)))
    psi[0][1:-1] = initial_state(x[1:-1], x0, p0, sigma, h)


    A = np.zeros(nx) + 0j
    B = np.zeros(nx) + 0j
    U = np.zeros(nx) + 0j
    R = np.zeros(nx) + 0j



    for i in range(0, len(t)-1):


        A[1:nx-1] = fun_A(x, t[i], dx, dt, m, h, V(x[1:nx-1]))
        B[1:nx-1] = fun_B(x, t[i], psi[i], dx, dt, m, h, V(x[1:nx-1]))

        U[1] = 1.0/A[1]
        for k in range(1, nx):
            U[k] = 1.0/(A[k] - U[k-1])

    #     U[1:nx-1] = np.divide(1.0, A[1:nx-1] - U[0:nx-2])


        R[1] = B[1]*U[1] 
        for k in range(1, nx):
            R[k] = (B[k] - R[k-1])*U[k]
    #     R[1:nx-1] = np.multiply((B[1:nx-1] - R[0:nx-2]), U[1:nx-1])

        psi[i+1][-1] = R[-1]  # check this

        for k in range(nx-2, 0, -1):
            psi[i+1][k] = R[k] - U[k]*psi[i+1][k+1]



    fig = plt.figure()
    ax1 = plt.axes(xlim=(0, nx*dx), ylim=(np.min(psi.real), np.max(psi.real)))
    line, = ax1.plot(x, psi[i-1].real, lw=2)

    
    def init():
        line.set_data(x, psi[i-1].real)
        return line,

    def animate(i):        
        line.set_data(x, psi[i].real)
        return line,

    ani = animation.FuncAnimation(fig, 
                                   animate, 
                                   init_func=init,
                                   frames=time_points, interval=40, blit=True)


    writervideo = animation.FFMpegWriter(fps=60) 
    ani.save('animation.mp4', writer=writervideo)
    
    
    import os
    
    def filename():
        filename = animation.mp4
        os.system("start " + filename)

    open(filename)







# Main code for GUI

### Added functionality for more input buttons would be needed

In [ ]:
# The code for changing pages was derived from: http://stackoverflow.com/questions/7546050/switch-between-two-frames-in-tkinter
# License: http://creativecommons.org/licenses/by-sa/3.0/	
# https://www.py4u.net/discuss/187096
    

LARGE_FONT= ("Verdana", 12)


class SeaofBTCapp(tk.Tk):

    def __init__(self, *args, **kwargs):
        
        tk.Tk.__init__(self, *args, **kwargs)

#         tk.Tk.iconbitmap(self, default="clienticon.ico")
        tk.Tk.wm_title(self, "Sea of BTC client")
        
        
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand = True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (StartPage, PageOne, PageTwo, PageThree):

            frame = F(container, self)

            self.frames[F] = frame

            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame(StartPage)

    def show_frame(self, cont):

        frame = self.frames[cont]
        frame.tkraise()

        
class StartPage(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Start Page", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button = ttk.Button(self, text="Visit Page 1",
                            command=lambda: controller.show_frame(PageOne))
        button.pack()

        button2 = ttk.Button(self, text="Visit Page 2",
                            command=lambda: controller.show_frame(PageTwo))
        button2.pack()

        button3 = ttk.Button(self, text="Graph Page",
                            command=lambda: controller.show_frame(PageThree))
        button3.pack()


class PageOne(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Page One!!!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home",
                            command=lambda: controller.show_frame(StartPage))
        button1.pack()

        button2 = ttk.Button(self, text="Page Two",
                            command=lambda: controller.show_frame(PageTwo))
        button2.pack()


class PageTwo(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Page Two!!!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home",
                            command=lambda: controller.show_frame(StartPage))
        button1.pack()

        button2 = ttk.Button(self, text="Page One",
                            command=lambda: controller.show_frame(PageOne))
        button2.pack()



class PageThree(tk.Frame):



    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Graph Page!", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home",
                            command=lambda: controller.show_frame(StartPage))
        button1.pack(side=tk.LEFT)
                    
        
        e = Entry(self, width=50)
        e.pack()
        e.insert(0, "What's your potential? (e.g. x*sin(x^2))")
        
        minx = Entry(self, width=50)
        minx.pack()
        minx.insert(0, "Min x")

        
        maxx = Entry(self, width=50)
        maxx.pack()
        maxx.insert(0, "Max x")



        fig = Figure(figsize=(5,5), dpi=100)
        a = fig.add_subplot(111)
        canvas=FigureCanvasTkAgg(fig,master=self)

                
        potentialButton = ttk.Button(self, text="Show Potential", 
                              command=lambda: Potential_Click2(canvas,a,
                                                               cmd=e.get(), 
                                                               min_x=minx.get(),
                                                               max_x=maxx.get()))
        potentialButton.pack()
        


        animButton = ttk.Button(self, text="Run Animation", 
                              command=lambda: Animation_Click(canvas,a,
                                                               cmd=e.get(), 
                                                             min_x=minx.get(),
                                                               max_x=maxx.get()))
        animButton.pack()

        
        

        canvas = FigureCanvasTkAgg(fig, self)
        canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

        toolbar = NavigationToolbar2Tk(canvas, self)
        toolbar.update()
        canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        

app = SeaofBTCapp()
app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/dash_env/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/17/p8qnv7p57d73kx2f_7bd35bm0000gn/T/ipykernel_54150/3538575173.py", line 139, in <lambda>
    command=lambda: Animation_Click(canvas,a,
  File "/var/folders/17/p8qnv7p57d73kx2f_7bd35bm0000gn/T/ipykernel_54150/2280218061.py", line 152, in Animation_Click
    open(filename)
TypeError: expected str, bytes or os.PathLike object, not function
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/dash_env/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/var/folders/17/p8qnv7p57d73kx2f_7bd35bm0000gn/T/ipykernel_54150/3538575173.py", line 139, in <lambda>
    command=lambda: Animation_Click(canvas,a,
  File "/var/folders/17/p8qnv7p57d73kx2f_7bd35bm0000gn/T/ipykernel_54150/2280218061.py

In [5]:
# from scipy.sparse import diags


# data_points = 1000
# nx = data_points + 1

# dt = 0.01
# time_points = 1000
# t = np.arange(0, time_points*dt, dt)

# dx = 0.01
# x = np.arange(0, nx*dx, dx)


# m = 5.68
# h = 0.6582
# x0 = nx*dx/2
# p0 = nx*dx/10
# sigma = nx*dx/20


# # lambdav = 1j/2

# psi = np.zeros((len(t), len(x))) + 0j
# # print("PSI shape = " + str(np.shape(psi)))
# psi[0][1:-1] = initial_state(x[1:-1], x0, p0, sigma, h)


# A = np.zeros(nx) + 0j
# B = np.zeros(nx) + 0j
# U = np.zeros(nx) + 0j
# R = np.zeros(nx) + 0j



# for i in range(0, len(t)-1):
    
    
#     A[1:nx-1] = fun_A(x, t[i], dx, dt, m, h)
#     B[1:nx-1] = fun_B(x, t[i], psi[i], dx, dt, m, h)
    
#     U[1] = 1.0/A[1]
#     for k in range(1, nx):
#         U[k] = 1.0/(A[k] - U[k-1])
    
    
#     R[1] = B[1]*U[1] 
#     for k in range(1, nx):
#         R[k] = (B[k] - R[k-1])*U[k]
        
#     psi[i+1][-1] = R[-1]  # check this
    
#     for k in range(nx-2, 0, -1):
#         psi[i+1][k] = R[k] - U[k]*psi[i+1][k+1]



# fig = plt.figure()
# ax = plt.axes(xlim=(0, nx*dx), ylim=(np.min(psi.real), np.max(psi.real)))
# line, = ax.plot(x, psi[i-1].real, lw=2)


# def init():
#     line.set_data([], [])
#     return line,

# def animate(i):        
#     line.set_data(x, psi[i].real)
#     return line,

# ani = animation.FuncAnimation(fig, 
#                                animate, 
#                                init_func=init,
#                                frames=time_points, interval=40, blit=True)


# writervideo = animation.FFMpegWriter(fps=60) 
# ani.save('animation.mp4', writer=writervideo)



